In [2]:
import random
import json
import functools as fn
import numpy as np
import string
import hashlib

charset = string.ascii_lowercase+string.digits+',. '
charset_idmap = {e: i for i, e in enumerate(charset)}

ksz = 80

In [67]:
def decrypt(ctx, key):
    N, ksz = len(charset), len(key)
    return ''.join(charset[(c-key[i % ksz]) % N] for i, c in enumerate(ctx))

def toPrintable(data):
    ul = ord('_')
    data = bytes(c if 32 <= c < 127 else ul for c in data)
    return data.decode('ascii')

with open('./output.txt') as f:
    ctx = f.readline().strip()[4:]
    enc = bytes.fromhex(f.readline().strip()[6:])
ctx = [charset_idmap[c] for c in ctx]

with open('./ngrams.json') as f:
    ngrams = json.load(f)


In [4]:
@fn.lru_cache(10000)
def get_trigram(x):
    x = ''.join(x)
    y = ngrams.get(x)
    if y is not None:
        return y
    ys = []
    a, b = ngrams.get(x[:2]), ngrams.get(x[2:])
    if a is not None and b is not None:
        ys.append(a+b)
    a, b = ngrams.get(x[:1]), ngrams.get(x[1:])
    if a is not None and b is not None:
        ys.append(a+b)
    if len(ys):
        return max(ys)
    if any(c not in ngrams for c in x):
        return -25
    return sum(map(ngrams.get, x))

@fn.lru_cache(10000)
def fitness(a):
    plain = decrypt(ctx, a)
    tgs = zip(plain, plain[1:], plain[2:])
    score = sum(get_trigram(tg) for tg in tgs)
    return score

def initialize(size):
    population = []
    for i in range(size):
        key = tuple(random.randrange(len(charset)) for _ in range(ksz))
        population.append(key)
    return population

def crossover(a, b, prob):
    r = list(a)
    for i in range(len(r)):
        if random.random() < prob:
            r[i] = b[i]
    return tuple(r)

def mutate(a):
    r = list(a)
    i = random.randrange(len(a))
    r[i] = random.randrange(len(charset))
    return tuple(r)

In [44]:
keys = np.array(initialize(7000))
scores = np.array([])
for i in keys:
    scores = np.append(scores, fitness(tuple(i)))


In [45]:
keys = keys[scores.argsort()[::-1]][:600]

In [ ]:
for m in range(2000):
    np.random.shuffle(keys)
    for i in range(len(keys)//2):
        child = np.array(crossover(keys[i*2], keys[i*2+1], 0.45))
        keys = np.concatenate((keys, [child]))
    np.random.shuffle(keys)
    for i in range(len(keys)//2):
        keys[i] = mutate(keys[i])
    scores = np.array([])
    for i in keys:
        scores = np.append(scores, fitness(tuple(i)))

    keys = keys[scores.argsort()[::-1]][:600]
    scores = scores[scores.argsort()[::-1]][:600]

    print(m, int(scores[0]), decrypt(ctx, keys[0]))

In [59]:
keys[0]
# print(charset_idmap)

array([24, 23, 23, 21, 12, 11, 12, 33,  9, 15, 37, 25, 20, 17, 36,  1, 26,
       31, 35, 17, 23, 11,  2, 19, 15, 28, 25,  8,  4, 31, 29, 21, 25, 24,
       19, 14, 32, 19, 16, 34, 27,  0, 28,  8, 21, 24, 21, 10, 21, 28,  4,
        2,  6, 32, 20, 33, 11, 10, 36, 34, 31, 30, 28, 12, 10,  2, 19, 27,
       38,  7,  0, 20, 29, 38, 27,  2, 21, 17,  1, 28])

In [61]:
x = 'gimli is a 384 bits permutation designed to achieve high security with high performance across a broad range of platforms. it is currently in round 2 of the nist lightweight cryptographic project, the submission consisting of an authenticated encryption and a cryptographic hash function. in this paper, we focus on the gimli cipher, which performs authenticated encryption with associated data.'
y = '45d6ukumip,ppi9c8loq9lt89iz1mgdu22.w 6u.0tdhv02szkibnb0bk2b,mi,58bc9so 1.f8b,vs7 bdvznq6jrpa 99bz6n5ukbapc5mdg8ub9t7.77hg.1qbx32u4ytx,7nw89df3g04pw01goz2s8gu4jhewc3ss5qnxe6aq slhp50yc.w,1htje430 l5 x 0sjj76a23drbih7mt2qdf,10pbtb.hua,dbv3tbi203zzn3sy8ga7q,o349qwy0.8d5zeh,31x0ol0pain413 8iu,rbza2mkz,k9izl6gs6nju 2nbbbyf145d6ukocywcdrqti87dq9lt13g.0d5kb6267bvqo5d1m80 8,imqt5dc4r98kjdosc 5cgduj z'

N = len(charset)
ksz = 80
tmp = []

for i in range(ksz):
    tmp.append((charset_idmap[y[i]] + N - charset_idmap[x[i]])%N)

In [72]:
k = hashlib.sha512(''.join(charset[k] for k in tmp).encode('ascii')).digest()
flag = bytes(ci ^ ki for ci, ki in zip(enc.ljust(len(k), b'\0'), k))
flag

b'FLAG{3c7166f852e3eaed71c81875e0c290562eff2c0}\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'